In [1]:
from icecream import ic
from ordered_set import OrderedSet
from itertools import product
import itertools as _itertools
import stim
from idtcorev2 import jacobian_coefficient_calc
from pygsti.circuits.circuit import Circuit
from pygsti.baseobjs.label import Label

# Update important parameters here!
VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

In [2]:
num_qubits = 2
max_weight = 2
term_dict = {("S", "XX"): 0.0001}

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [3]:
HS_index_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=1, max_weight=max_weight
)

pauli_node_attributes = list([p for p in HS_index_iterator])
ca_pauli_node_attributes = list(_itertools.combinations(pauli_node_attributes, 2))

def ca_pauli_weight_filter(pauli_pair, max_weight):
    used_indices_1 = set(
        i for i, ltr in enumerate(str(pauli_pair[0])[1:]) if ltr != "_"
    )
    used_indices_2 = set(
        i for i, ltr in enumerate(str(pauli_pair[1])[1:]) if ltr != "_"
    )
    intersect = used_indices_1.intersection(used_indices_2)
    if len(intersect) > 0 and len(intersect) <= max_weight:
        return True

ca_pauli_node_attributes = [
    ppair
    for ppair in ca_pauli_node_attributes
    if ca_pauli_weight_filter(ppair, max_weight)
]

measure_string_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=num_qubits
)
prep_string_iterator = product([1,-1],[p for p in measure_string_iterator])
measure_string_attributes = list([p for p in measure_string_iterator])
prep_string_attributes = list(a*b for a,b in prep_string_iterator)
prep_meas_pair = list(product(prep_string_attributes, measure_string_attributes))

In [4]:
hs_error_gen_classes = "hs"
ca_error_gen_classes = ""


hs_experiment = list(
    product(
        hs_error_gen_classes,
        pauli_node_attributes,
        measure_string_attributes,
        measure_string_attributes,
    )
)
ca_experiment = list(
    product(
        ca_error_gen_classes,
        ca_pauli_node_attributes,
        measure_string_attributes,
        measure_string_attributes,
    )
)



import pandas as pd

# df = pd.DataFrame()
jacobian_coef_dict = {"index": OrderedSet(), "columns": OrderedSet()}
data = {}

# These come back as class, index, prep_str, meas_str, observ_str: coef
# I THINK this is right, should double check and write unit tests
for key in hs_experiment + ca_experiment:
    elt = jacobian_coefficient_calc(*key)
    for el in elt:
        if el:
            observable = ",".join(str(s)[:] for s in el[-4:-1])
            egen = ",".join(str(s) for s in el[:-4])
            coef = int(el[-1])
            jacobian_coef_dict["index"].add(observable)
            jacobian_coef_dict["columns"].add(egen)
            if data.get(egen):
                data[egen].append(coef)
            else:
                data[egen] = [coef]

df = pd.DataFrame(data, index= jacobian_coef_dict["index"], columns=jacobian_coef_dict["columns"])

In [5]:
df

,"H,+X_","H,+Y_","H,+Z_","H,+_X","H,+_Y","H,+_Z","H,+XX","H,+XY","H,+XZ","H,+YX",...,"S,+_Z","S,+XX","S,+XY","S,+XZ","S,+YX","S,+YY","S,+YZ","S,+ZX","S,+ZY","S,+ZZ"
"+XX,+XX,+X_",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,-2,-2,-2,-2,-2,-2
"+XX,+XX,+_X",0,0,0,0,0,0,0,0,0,0,...,-2,0,-2,-2,0,-2,-2,0,-2,-2
"+XX,+XX,+XX",0,0,0,0,0,0,0,0,0,0,...,-2,0,-2,-2,-2,0,0,-2,0,0
"+XX,+XY,+X_",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,-2,-2,-2,-2,-2,-2
"+XX,+XY,+_Y",0,0,0,0,0,2,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"+ZZ,+ZY,+_Y",0,0,0,-2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"+ZZ,+ZY,+ZY",0,0,0,-2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"+ZZ,+ZZ,+Z_",0,0,0,0,0,0,0,0,0,0,...,0,-2,-2,-2,-2,-2,-2,0,0,0
"+ZZ,+ZZ,+_Z",0,0,0,0,0,0,0,0,0,0,...,0,-2,-2,0,-2,-2,0,-2,-2,0


In [6]:
import numpy as np
np.set_printoptions(precision=1, linewidth=1000)

In [7]:
import pygsti
from pygsti.extras import idletomography as idt
from pygsti.baseobjs import Label
gates = ["Gi", "Gx", "Gy", "Gcnot"]
max_lengths = [1, 2]
pspec = pygsti.processors.QubitProcessorSpec(
        num_qubits, gates, geometry="line", nonstd_gate_unitaries={(): num_qubits, "Gi": np.eye((2**num_qubits))},
        availability={"Gi": [tuple(i for i in range(num_qubits))]},
    )
mdl_target = pygsti.models.create_crosstalk_free_model(pspec)
paulidicts = idt.determine_paulidicts(mdl_target)
global_idle_string = [Label("Gi", tuple(i for i in range(num_qubits)))]
idle_experiments = idt.make_idle_tomography_list(
        num_qubits, max_lengths, paulidicts, maxweight=1, idle_string=global_idle_string
    )
idle_experiments

ic| fidpairs: [(State[+X+X], State[+X+X]),
               (State[+X+X], State[+X+Y]),
               (State[+X+X], State[+X+Z]),
               (State[+X+X], State[+Y+X]),
               (State[+X+X], State[+Y+Y]),
               (State[+X+X], State[+Y+Z]),
               (State[+X+X], State[+Z+X]),
               (State[+X+X], State[+Z+Y]),
               (State[+X+X], State[+Z+Z]),
               (State[+X+Y], State[+X+X]),
               (State[+X+Y], State[+X+Y]),
               (State[+X+Y], State[+X+Z]),
               (State[+X+Y], State[+Y+X]),
               (State[+X+Y], State[+Y+Y]),
               (State[+X+Y], State[+Y+Z]),
               (State[+X+Y], State[+Z+X]),
               (State[+X+Y], State[+Z+Y]),
               (State[+X+Y], State[+Z+Z]),
               (State[+X+Z], State[+X+X]),
               (State[+X+Z], State[+X+Y]),
               (State[+X+Z], State[+X+Z]),
               (State[+X+Z], State[+Y+X]),
               (State[+X+Z], State[+Y+Y]),
           

[Circuit([Gy:0Gy:1]Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1[Gy:0Gx:1]Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1[Gy:0Gx:1]Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gy:0Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1Gy:0Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1[Gx:0Gy:1]Gy:1Gy:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1[Gx:0Gy:1]Gy:1Gy:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1[Gx:0Gx:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1[Gx:0Gx:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gx:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1Gx:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gy:1Gy:1Gy:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1Gy:1Gy:1Gy:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gx:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1Gx:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1@(0,1)),
 Circuit([Gy:0Gx:1]Gx:1Gx:1Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gx:1]Gx:1Gx:1Gi:0:1Gi:0:1[Gy:0Gy

In [8]:
mdl_datagen = pygsti.models.create_crosstalk_free_model(
pspec, lindblad_error_coeffs={"Gi": term_dict},lindblad_parameterization="GLND")

# Error models! Random with right CP constraints from Taxonomy paper
ds = pygsti.data.simulate_data(
    mdl_datagen, idle_experiments, 10000000, seed=8675309, sample_error="none"
)

In [9]:
from idttools import allerrors, all_full_length_observables, alloutcomes
import collections as _collections
from pygsti.circuits.circuit import Circuit as _Circuit
def report_observed_rates(nqubits,
    dataset,
    max_lengths,
    pauli_basis_dicts,
    maxweight=2,
    idle_string=global_idle_string):
    
    all_fidpairs = dict(enumerate(idt.idle_tomography_fidpairs(nqubits)))
    ic(all_fidpairs)
    if nqubits == 1:  # special case where line-labels may be ('*',)
        if len(dataset) > 0:
            first_circuit = list(dataset.keys())[0]
            line_labels = first_circuit.line_labels
        else:
            line_labels = (0,)
        GiStr = _Circuit(idle_string, line_labels=line_labels)
    else:
        GiStr = _Circuit(idle_string, num_lines=nqubits)
    obs_infos = dict()
    errors = allerrors(nqubits, maxweight)
    fit_order = 1
    observed_error_rates = {}
    obs_error_rates_by_exp = {}
    whatever = {}
    for ifp, pauli_fidpair in all_fidpairs.items():
        all_outcomes = idt.idttools.allobservables(pauli_fidpair[1], maxweight)
        infos_for_this_fidpair = _collections.OrderedDict()
        ic(pauli_fidpair)
        for j, out in enumerate(all_outcomes):
            info = idt.compute_observed_err_rate(
                dataset,
                pauli_fidpair,
                pauli_basis_dicts,
                GiStr,
                out,
                max_lengths,
                fit_order,
            )

            infos_for_this_fidpair[out] = info
            
            obs_infos[ifp] = infos_for_this_fidpair
            observed_error_rates[ifp] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ]
            obs_error_rates_by_exp[str(pauli_fidpair[0]).replace("+",""), str(pauli_fidpair[1]).replace("+",""), str(out).replace("I","_")] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ][-1]
        whatever[pauli_fidpair] = 1
    return observed_error_rates, obs_error_rates_by_exp

In [10]:
observed_error_rates, obs_error_rates_by_exp = report_observed_rates(num_qubits, ds, max_lengths, paulidicts)

ic| fidpairs: [(State[+X+X], State[+X+X]),
               (State[+X+X], State[+X+Y]),
               (State[+X+X], State[+X+Z]),
               (State[+X+X], State[+Y+X]),
               (State[+X+X], State[+Y+Y]),
               (State[+X+X], State[+Y+Z]),
               (State[+X+X], State[+Z+X]),
               (State[+X+X], State[+Z+Y]),
               (State[+X+X], State[+Z+Z]),
               (State[+X+Y], State[+X+X]),
               (State[+X+Y], State[+X+Y]),
               (State[+X+Y], State[+X+Z]),
               (State[+X+Y], State[+Y+X]),
               (State[+X+Y], State[+Y+Y]),
               (State[+X+Y], State[+Y+Z]),
               (State[+X+Y], State[+Z+X]),
               (State[+X+Y], State[+Z+Y]),
               (State[+X+Y], State[+Z+Z]),
               (State[+X+Z], State[+X+X]),
               (State[+X+Z], State[+X+Y]),
               (State[+X+Z], State[+X+Z]),
               (State[+X+Z], State[+Y+X]),
               (State[+X+Z], State[+Y+Y]),
           

In [11]:
obs_rats = [v for v in obs_error_rates_by_exp.values()]

In [12]:
df.index

Index(['+XX,+XX,+X_', '+XX,+XX,+_X', '+XX,+XX,+XX', '+XX,+XY,+X_',
       '+XX,+XY,+_Y', '+XX,+XY,+XY', '+XX,+XZ,+X_', '+XX,+XZ,+_Z',
       '+XX,+XZ,+XZ', '+XX,+YX,+Y_',
       ...
       '+ZZ,+YZ,+YZ', '+ZZ,+ZX,+Z_', '+ZZ,+ZX,+_X', '+ZZ,+ZX,+ZX',
       '+ZZ,+ZY,+Z_', '+ZZ,+ZY,+_Y', '+ZZ,+ZY,+ZY', '+ZZ,+ZZ,+Z_',
       '+ZZ,+ZZ,+_Z', '+ZZ,+ZZ,+ZZ'],
      dtype='object', length=243)

In [13]:
obs_error_rates_by_exp

{('XX', 'XX', ' X_'): -5.1730544255886464e-17,
 ('XX', 'XX', ' _X'): -5.1730544255886464e-17,
 ('XX', 'XX', ' XX'): -5.1730544255886464e-17,
 ('XX', 'XY', ' X_'): -5.1730544255886464e-17,
 ('XX', 'XY', ' _Y'): 0.0,
 ('XX', 'XY', ' XY'): 0.0,
 ('XX', 'XZ', ' X_'): -5.1730544255886464e-17,
 ('XX', 'XZ', ' _Z'): 0.0,
 ('XX', 'XZ', ' XZ'): 0.0,
 ('XX', 'YX', ' Y_'): 0.0,
 ('XX', 'YX', ' _X'): -5.1730544255886464e-17,
 ('XX', 'YX', ' YX'): 0.0,
 ('XX', 'YY', ' Y_'): 0.0,
 ('XX', 'YY', ' _Y'): 0.0,
 ('XX', 'YY', ' YY'): 0.0,
 ('XX', 'YZ', ' Y_'): 0.0,
 ('XX', 'YZ', ' _Z'): 0.0,
 ('XX', 'YZ', ' YZ'): 0.0,
 ('XX', 'ZX', ' Z_'): 0.0,
 ('XX', 'ZX', ' _X'): -5.1730544255886464e-17,
 ('XX', 'ZX', ' ZX'): 0.0,
 ('XX', 'ZY', ' Z_'): 0.0,
 ('XX', 'ZY', ' _Y'): 0.0,
 ('XX', 'ZY', ' ZY'): 0.0,
 ('XX', 'ZZ', ' Z_'): 0.0,
 ('XX', 'ZZ', ' _Z'): 0.0,
 ('XX', 'ZZ', ' ZZ'): 0.0,
 ('XY', 'XX', ' X_'): -5.1730544255886464e-17,
 ('XY', 'XX', ' _X'): 0.0,
 ('XY', 'XX', ' XX'): 0.0,
 ('XY', 'XY', ' X_'): 1.307045

In [14]:
j = df.to_numpy()

In [15]:
jinv = np.linalg.pinv(j)

In [16]:
intrins_errs = jinv @ obs_rats

# Supposed intrinsic error rates (wrong)

In [17]:
dict(zip(df.columns,intrins_errs))

{'H,+X_': -4.925942624761042e-21,
 'H,+Y_': 6.123001744652034e-21,
 'H,+Z_': -7.976885787133889e-21,
 'H,+_X': 4.3579509418436755e-20,
 'H,+_Y': -2.443281749763007e-20,
 'H,+_Z': -7.553595569867582e-21,
 'H,+XX': -5.1060948832336865e-20,
 'H,+XY': 5.720772489908759e-20,
 'H,+XZ': -1.2220733556938211e-20,
 'H,+YX': -1.3576593173058827e-20,
 'H,+YY': 2.1010656192015786e-20,
 'H,+YZ': 1.1708864984427557e-20,
 'H,+ZX': 3.103372579479225e-22,
 'H,+ZY': 2.912308211933502e-21,
 'H,+ZZ': 1.9404481198950892e-20,
 'S,+X_': -1.115891932877761e-17,
 'S,+Y_': 3.1872959680681243e-18,
 'S,+Z_': 3.2953816812928555e-18,
 'S,+_X': -1.12102964749837e-17,
 'S,+_Y': 3.2045904962156947e-18,
 'S,+_Z': 3.297075747187364e-18,
 'S,+XX': 9.997001432101552e-05,
 'S,+XY': 3.183573332255288e-18,
 'S,+XZ': 3.2059708303205738e-18,
 'S,+YX': 3.2055961888838996e-18,
 'S,+YY': 1.4674845811180753e-18,
 'S,+YZ': 1.43730762984109e-18,
 'S,+ZX': 3.187357804208084e-18,
 'S,+ZY': 1.3640591791239191e-18,
 'S,+ZZ': 1.4446146915